In [6]:
from azureml.core import Workspace, Experiment

# Configure experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='cifar10_cnn_remote')

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# cluster configuration
cluster_name = "mldemocompute"
min_nodes = 0
max_nodes = 2
vm_size = "STANDARD_D2_V2"

# verify that the cluster exists already
try:
    aml_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print('Cluster not `%s` not found, creating one now.' % cluster_name)
    config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=min_nodes, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=config)

# wait until the cluster is ready
aml_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
import os
# define the execution script location
script = 'cifar10_cnn_remote.py'
script_folder = os.path.join(os.getcwd(), 'code')

# define the PIP requirements file location
pipreq = 'requirements.txt'
pipreq_path = os.path.join(os.getcwd(), '.azureml', pipreq)


In [9]:
from azureml.core.environment import Environment

# set a remote environment from a PIP requirement file
myenv = Environment.from_pip_requirements(name = "remote_env", file_path = pipreq_path)
# show environment object
myenv

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "remote_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
          

In [10]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, 
                      script=script, 
                      compute_target = aml_cluster, 
                      environment = myenv)
run = exp.submit(src)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…